In [12]:
from pulp import *
import pandas as pd
import numpy as np

In [19]:
# Problem optymalizacyjny, Plan pracy
# Uzupełnić plan pracy z uwzględnieniem dostępności pracowników oraz wynagrodzenia za godzinę pracy dla danego pracownika
# Cel - jak najmniejszy koszt dla pracodawcy

prob = LpProblem("koszt",LpMinimize)

# Stworzenie dla każdej osoby zmiennej dla każdego dnia pracy - 8 osób * 6 dni

# Osoba    Dostepność[h]     Cena za godzine pracy
# Igor          4                     170
# Marcin        2                      60
# Franek        2                      80
# Piotr         8                     200
# Ania          8                      90
# Zenon         2                      10
# Filip         8                      80
# Kuba          6                      60

# Plan dla sklepu: 
# pon = 8
# wt = 4
# sr = 4
# czw = 4
# pt = 8
# sob = 2

prob = LpProblem("Scheduling_problem", LpMinimize)

dni = ["Poniedziałek", "Wtorek", "Środa", "Czwartek", "Piątek", "Sobota", "Niedziela"]
osoby = ["Igor", "Marcin", "Franek", "Piotr", "Ania", "Zenon","Filip","Kuba"]
stawka = [170,60,80,200,90,10,80,60] # Stawka godzinowa osób.
dostepnosc = [4,2,2,8,8,2,8,6] # Dostępność godzin osób tygodniowo.
godzinyOtwarcia = [8,4,4,4,8,2,0] # Ilość godzin do wypracowania w ciągu każdego dnia.


j=0
stawki={}
for i in osoby:
    stawki[i]=stawka[j]
    j+=1

In [20]:
# Stworzenie zmiennych potrzebnych do roziwązania problemu, funkcji celu, dodatkowych ograniczeń
prob += (pOSOBA1+wOSOBA1+sOSOBA1+czwOSOBA1+ptOSOBA1+sobOSOBA1)*170+(pOSOBA2+wOSOBA2+sOSOBA2+czwOSOBA2+ptOSOBA2 + sobOSOBA2)*60+(pOSOBA3+wOSOBA3+sOSOBA3+czwOSOBA3+ptOSOBA3+sobOSOBA3)*80+(pOSOBA4+wOSOBA4+sOSOBA4+czwOSOBA4+ptOSOBA4+sobOSOBA4)*200+(pOSOBA5+wOSOBA5+sOSOBA5+czwOSOBA5+ptOSOBA5+sobOSOBA5)*90+(pOSOBA6+wOSOBA6+sOSOBA6+czwOSOBA6+ptOSOBA6+sobOSOBA6)*18+(pOSOBA7+wOSOBA7+sOSOBA7+czwOSOBA7+ptOSOBA7+sobOSOBA7)*100+(pOSOBA8+wOSOBA8+sOSOBA8+czwOSOBA8+ptOSOBA8+sobOSOBA8)*100


czas_otwarcia_dni = LpVariable.dicts("czas_otwarcia",((i) for i in dni), lowBound=0, cat="Integer")
czas_dostepnosc_tyg = LpVariable.dicts("dostepnosc",((i) for i in osoby), lowBound=0, upBound=((j) for j in dostepnosc),cat="Integer")
czas_pracy_pon = LpVariable.dicts("czaspracy_pon",((i) for i in osoby), lowBound=0,cat="Integer")
czas_pracy_wt = LpVariable.dicts("czaspracy_wt",((i) for i in osoby), lowBound=0,cat="Integer")
czas_pracy_sr = LpVariable.dicts("czaspracy_sr",((i) for i in osoby), lowBound=0,cat="Integer")
czas_pracy_cz = LpVariable.dicts("czaspracy_cz",((i) for i in osoby), lowBound=0,cat="Integer")
czas_pracy_pt = LpVariable.dicts("czaspracy_pt",((i) for i in osoby), lowBound=0,cat="Integer")
czas_pracy_sob = LpVariable.dicts("czaspracy_sob",((i) for i in osoby), lowBound=0,cat="Integer")
czas_pracy_nd = LpVariable.dicts("czaspracy_nd",((i) for i in osoby), lowBound=0,cat="Integer")


#funkcja celu czyli koszt pracy każdej osoby w tyg -> to minimalizujemy
prob+=(pulp.lpSum([(czas_pracy_pon[i]+czas_pracy_wt[i]+czas_pracy_sr[i]+czas_pracy_cz[i]+czas_pracy_pt[i]+czas_pracy_sob[i]+czas_pracy_nd[i]) *stawki[i]] for i in osoby ))


#laczna suma pracy osob w każdym dniu rowna dlugosci otwarcia sklepu
prob+=pulp.lpSum([czas_pracy_pon[i] for i in osoby])==godzinyOtwarcia[0]
prob+=pulp.lpSum([czas_pracy_wt[i] for i in osoby])==godzinyOtwarcia[1]
prob+=pulp.lpSum([czas_pracy_sr[i] for i in osoby])==godzinyOtwarcia[2]
prob+=pulp.lpSum([czas_pracy_cz[i] for i in osoby])==godzinyOtwarcia[3]
prob+=pulp.lpSum([czas_pracy_pt[i] for i in osoby])==godzinyOtwarcia[4]
prob+=pulp.lpSum([czas_pracy_sob[i] for i in osoby])==godzinyOtwarcia[5]
prob+=pulp.lpSum([czas_pracy_nd[i] for i in osoby])==godzinyOtwarcia[6]

# dodaje ograniczenie w postaci suma czasu pracy osoby z tygodnia ma byc mniejsza bądź równa dostępności tej osoby
j=0
for i in osoby:
    prob+=czas_pracy_pon[i]+czas_pracy_wt[i]+czas_pracy_sr[i]+czas_pracy_cz[i]+czas_pracy_pt[i]+czas_pracy_sob[i]+czas_pracy_nd[i]<=dostepnosc[j]
    j+=1
    
    
prob.solve()

1

In [21]:
#Drukowanie informacji o problemie, liczbie godzin i dniu pracy
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    if v.varValue != 0:
            if (v.name.find("_pon")>0):
                print(v.name, "=", v.varValue)
for v in prob.variables():
    if v.varValue != 0:
            if (v.name.find("_wt")>0):
                print(v.name, "=", v.varValue)             
for v in prob.variables():
    if v.varValue != 0:
            if (v.name.find("_sr")>0):
                print(v.name, "=", v.varValue)  
for v in prob.variables():
    if v.varValue != 0:
            if (v.name.find("_cz")>0):
                print(v.name, "=", v.varValue)   
for v in prob.variables():
    if v.varValue != 0:
            if (v.name.find("_pt")>0):
                print(v.name, "=", v.varValue) 
for v in prob.variables():
    if v.varValue != 0:
            if (v.name.find("_sob")>0):
                print(v.name, "=", v.varValue)   
for v in prob.variables():
    if v.varValue != 0:
            if (v.name.find("_nd")>0):
                print(v.name, "=", v.varValue)                  
print("\nKoszt zatrudnienia studentów to {} zł.  ".format( value(prob.objective)))

Status: Optimal
czaspracy_pon_Igor = 2.0
czaspracy_pon_Kuba = 4.0
czaspracy_pon_Marcin = 2.0
czaspracy_wt_Franek = 2.0
czaspracy_wt_Kuba = 2.0
czaspracy_sr_Filip = 2.0
czaspracy_sr_Zenon = 2.0
czaspracy_cz_Filip = 4.0
czaspracy_pt_Ania = 8.0
czaspracy_sob_Filip = 2.0

Koszt zatrudnienia studentów to 2360.0 zł.  
